Import

In [9]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter
from scipy.stats import zscore
from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt

import seaborn as sns 
import bk.load
import bk.compute
import bk.plot
import bk.signal
import bk.stats

import os


import astropy.stats as apy
plt.rcParams['svg.fonttype'] = 'none'

In [10]:
def fr(spikes,state):
    return(len(spikes.restrict(state))/state.tot_length(time_units='s'))

def mod_stats(spikes,phases,nbins):
    phases_signal_distrib, _ = np.histogram(phases.values,nbins,density=True)
    phases_signal_distrib = phases_signal_distrib*nbins / np.sum(phases_signal_distrib)


    neuron_phase = phases.realign(spikes)
    
    stats = {
        'pvalue':np.nan,
        'kappa':np.nan,
        'MRL':np.nan
    }
    
    if len(neuron_phase) == 0: return stats

    phases_distribution, bin_p = np.histogram(neuron_phase.values, nbins) 
    phases_distribution = phases_distribution/phases_signal_distrib

    bin_p = np.convolve(bin_p, [.5, .5], 'same')[1::]


    stats['pvalue'] = bk.stats.rayleigh(bin_p,weights = phases_distribution)
    stats['kappa'] = bk.compute.concentration(neuron_phase.values)
    stats['MRL'] = bk.compute.mean_resultant_length(bin_p,weights = phases_distribution)

    return bin_p,phases_distribution,stats
    
def phase_mod(neurons,metadata,state,low,high,nbins = 1000,channel=None):
    
    if low >50:
        return phase_mod_local(neurons,metadata,state,low,high,nbins = 1000)
            
    lfp = bk.load.lfp_in_intervals(channel,state)
    lfp_filt = bk.signal.passband(lfp,low,high,1250,2)
    power, phase = bk.signal.hilbert(lfp_filt)

    stats = {'fr_rem': [],
             'pvalue': [],
             'kappa': [],
             'MRL': [],
             'modulated':[]}
    for i,n in tqdm(enumerate(neurons)):
        bin_p, phases_distributions,stat = mod_stats(n.restrict(state),phase,nbins)
        stats['fr_rem'].append(fr(n,state))
        for k, s in stat.items():
            stats[k].append(s)
        if (stat['pvalue'] < 0.01) and stat['MRL'] > 0.04:
            stats['modulated'].append(True)
        else:
            stats['modulated'].append(False)

    phase_lock = pd.DataFrame({'FR_Rem': stats['fr_rem'],
                               'MRL': stats['MRL'],
                               'Kappa': stats['kappa'],
                               'pValue': stats['pvalue'],
                               'modulated': stats['modulated']})
    
        
    return bin_p,phases_distributions,phase_lock


def phase_mod_local(neurons, metadata, state, low, high, nbins=1000):

    stats = {'fr_rem': [],
                'pvalue': [],
                'kappa': [],
                'MRL': [],
                'modulated':[]}
    phases_distributions = np.zeros((nbins, len(neurons)))

    previous_channel = 0

    for i, (n, m) in tqdm(enumerate(zip(neurons, metadata.iloc))):
        shank_neighbour = bk.load.shank_neighbours(m.Shank)

        if np.isfinite(shank_neighbour['medial']):
            neighbour_channel = bk.load.best_channel(shank_neighbour['medial'])
        else:
            neighbour_channel = bk.load.best_channel(
                shank_neighbour['lateral'])

        if previous_channel != neighbour_channel:
            print(
                f'Loading a New LFP for neurons with shank : {m.Shank} --> {shank_neighbour}')
            previous_channel = neighbour_channel

            lfp = bk.load.lfp_in_intervals(neighbour_channel, state)
            lfp_filt = bk.signal.passband(lfp, low, high)
            power, phases = bk.signal.hilbert(lfp_filt)
            # phases_signal_distrib, _ = np.histogram(
            #     phases.values, nbins, density=True)
            # phases_signal_distrib = phases_signal_distrib * \
            #     nbins / np.sum(phases_signal_distrib)

        bin_p, phases_distributions[:, i], stat = mod_stats(n, phases, nbins)


        stats['fr_rem'].append (len(n.restrict(state))/state.tot_length(time_units='s'))

        for k, s in stat.items():
            stats[k].append(s)

        if (stat['pvalue'] < 0.01) and stat['MRL'] > 0.04:
            stats['modulated'].append(True)
        else:
            stats['modulated'].append(False)

    phase_lock = pd.DataFrame({'FR_Rem': stats['fr_rem'],
                                'MRL': stats['MRL'],
                                'Kappa': stats['kappa'],
                                'pValue': stats['pvalue'],
                                'modulated': stats['modulated']})

    return bin_p, phases_distributions, phase_lock


In [11]:
def plot_phase_mod(phases, mod, metadata, ax=None):
    if ax is None:
        fig, ax = plt.subplots(1, 1)

    ax.bar(phases, mod, phases[1]-phases[0], color='lightblue')
    ax.bar(phases+2*np.pi, mod, phases[1]-phases[0], color='lightblue')

    plt.suptitle(
        f'Neuron #{metadata.name}, Shank = {metadata.Shank}, Region = {metadata.Region}, Type = {metadata.Type}  \nFR_Rem = {metadata.FR_Rem} Hz \npValue = {metadata.pValue} \n MRL = {metadata.MRL}, Kappa = {metadata.Kappa} \n Modulated : {metadata.modulated}')
    if metadata.modulated:
        # ax.set_facecolor('lightgreen')
        fig.set_facecolor('lightgreen')
    plt.tight_layout()

In [12]:
def original_phasemod(neurons,metadata,state,low,high,nbins = 1000,channel=None):
    lfp = bk.load.lfp_in_intervals(channel,state)
    lfp_filt = bk.signal.passband(lfp,low,high,1250,2)
    power, phase = bk.signal.hilbert(lfp_filt)


    phases_signal_distrib, _ = np.histogram(phase.values,nbins,density=True)
    phases_signal_distrib = phases_signal_distrib*nbins / np.sum(phases_signal_distrib)


    neurons_phase = []
    neurons_rem_fr = []
    for i, n in tqdm(enumerate(neurons)):
        neurons_phase.append(phase.realign(n.restrict(state)))
        neurons_rem_fr.append(
            len(n.restrict(state))/state.tot_length(time_units='s'))

    pvalues = []
    kappas = []
    modulated = []
    MRLs = []
    phases_distributions = np.zeros((nbins,len(neurons)))
    for i, p in tqdm(enumerate(neurons_phase)):
        
        if len(p) == 0:
            pvalues.append(np.nan)
            kappas.append(np.nan)
            modulated.append(np.nan)
            MRLs.append(np.nan)
            continue


        phases_distributions[:,i], bin_p = np.histogram(neurons_phase[i].values, nbins) 
        phases_distributions[:,i] = phases_distributions[:,i]/phases_signal_distrib
        bin_p = np.convolve(bin_p, [.5, .5], 'same')[1::]


        pvalue = bk.stats.rayleigh(bin_p,weights = phases_distributions[:,i])
        pvalues.append(pvalue)

        kappa = bk.compute.concentration(p.values)
        kappas.append(kappa)


        MRL = bk.compute.mean_resultant_length(bin_p,weights = phases_distributions[:,i])
        MRLs.append(MRL)
        if (pvalue < 0.01) and MRL > 0.04:
            modulated.append(True)
        else:
            modulated.append(False)



    phase_lock = pd.DataFrame({'FR_Rem': neurons_rem_fr,
                            'MRL': MRLs,
                            'Kappa': kappas,
                            'pValue': pvalues,
                            'modulated': modulated})


    return bin_p,phases_distributions,phase_lock
    

In [13]:
def phase_mod_legacy(neurons,metadata,state,low,high,nbins = 1000,channel=None):

    if low >50:
        return phase_mod_local(neurons,metadata,state,low,high,nbins = 1000)
    
    lfp = bk.load.lfp_in_intervals(channel,state)
    lfp_filt = bk.signal.passband(lfp,low,high,1250,2)
    power, phase = bk.signal.hilbert(lfp_filt)

    # stats = {'fr_rem': [],
    #          'pvalue': [],
    #          'kappa': [],
    #          'MRL': [],
    #          'modulated':[]}
    # for i,n in tqdm(enumerate(neurons)):
    #     bin_p, phases_distributions,stat = mod_stats(n,phase,nbins)
    #     stats['fr_rem'].append(fr(n,state))
    #     for k, s in stat.items():
    #         stats[k].append(s)
    #     if (stat['pvalue'] < 0.01) and stat['MRL'] > 0.04:
    #         stats['modulated'].append(True)
    #     else:
    #         stats['modulated'].append(False)

    # phase_lock = pd.DataFrame({'FR_Rem': stats['fr_rem'],
    #                            'MRL': stats['MRL'],
    #                            'Kappa': stats['kappa'],
    #                            'pValue': stats['pvalue'],
    #                            'modulated': stats['modulated']})
    
        
    # return bin_p,phases_distributions,phase_lock


    phases_signal_distrib, _ = np.histogram(phase.values,nbins,density=True)
    phases_signal_distrib = phases_signal_distrib*nbins / np.sum(phases_signal_distrib)


    neurons_phase = []
    neurons_rem_fr = []
    for i, n in tqdm(enumerate(neurons)):
        neurons_phase.append(phase.realign(n.restrict(state)))
        neurons_rem_fr.append(
            len(n.restrict(state))/state.tot_length(time_units='s'))

    pvalues = []
    pvalues_bis = []
    kappas = []
    modulated = []
    modulated_bis = []
    MRLs = []
    MRLs_bis = []
    phases_distributions = np.zeros((nbins,len(neurons)))
    for i, p in tqdm(enumerate(neurons_phase)):
        
        if len(p) == 0:
            pvalues.append(np.nan)
            pvalues_bis.append(np.nan)
            kappas.append(np.nan)
            modulated.append(np.nan)
            modulated_bis.append(np.nan)
            MRLs.append(np.nan)
            MRLs_bis.append(np.nan)
            continue


        phases_distributions[:,i], bin_p = np.histogram(neurons_phase[i].values, nbins) 
        phases_distributions[:,i] = phases_distributions[:,i]/phases_signal_distrib
        bin_p = np.convolve(bin_p, [.5, .5], 'same')[1::]


        pvalue = apy.rayleightest(p.values)
        pvalues.append(pvalue)


        pvalue_bis = bk.stats.rayleigh(bin_p,weights = phases_distributions[:,i])
        pvalues_bis.append(pvalue_bis)

        kappa = bk.compute.concentration(p.values)
        kappas.append(kappa)

        MRL = bk.compute.mean_resultant_length(p.values)

        MRL_bis = bk.compute.mean_resultant_length(bin_p,weights = phases_distributions[:,i])
        MRLs.append(MRL)
        MRLs_bis.append(MRL_bis)
        if (pvalue < 0.01) and MRL > 0.04:
            modulated.append(True)
        else:
            modulated.append(False)

        if(pvalue_bis < 0.01) and MRL_bis > 0.04:
            modulated_bis.append(True)
        else:
            modulated_bis.append(False)



    phase_lock = pd.DataFrame({'FR_Rem': neurons_rem_fr,
                            'MRL': MRLs,
                            'MRL_bis':MRLs_bis,
                            'Kappa': kappas,
                            'pValue': pvalues,
                            'pValue_bis':pvalues_bis,
                            'modulated': modulated,
                            'modulated_bis':modulated_bis})

    
    # fig, ax = plt.subplots(1, 1)
    # ax.bar(bin_p, phases_signal_distrib, bin_p[1]-bin_p[0], color='lightblue')
    # ax.bar(bin_p+2*np.pi, phases_signal_distrib, bin_p[1]-bin_p[0], color='lightblue')

    return bin_p,phases_distributions,phase_lock


In [26]:
def main(base_folder,local_path,*args,**kwargs):
    bk.load.current_session_linux(base_folder,local_path)



    if os.path.exists('Analysis/Thetamod/hpc_theta_phase_lock.csv'):
        metadata_phaselock = pd.read_csv('Analysis/Thetamod/hpc_theta_phase_lock.csv')
        (phases,phases_modulations) = np.load('Analysis/Thetamod/hpc_theta_phases_mod.npy')

        return (phases,phases_modulations), metadata_phaselock

    
    states = bk.load.states()
    neurons, metadata = bk.load.spikes()
    phases,phases_modulations,metadata_phaselock = phase_mod(neurons,metadata,states['Rem'],4,12,channel=bk.load.ripple_channel())
    metadata_phaselock = pd.concat([metadata, metadata_phaselock], axis=1)

    np.save('Analysis/Thetamod/hpc_theta_phases_mod.npy',(phases,phases_modulations))
    metadata_phaselock.to_csv('Analysis/Thetamod/hpc_theta_phase_lock.csv', index = False)

    return (phases,phases_modulations), metadata_phaselock


In [27]:
(_,_),df = main('/mnt/electrophy/Gabrielle/GG-Dataset-Light/','Rat08/Rat08-20130713')

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /mnt/electrophy/Gabrielle/GG-Dataset-Light/Rat08/Rat08-20130713


In [28]:
def merge(batch):
    phases_modulations_all = []
    metadata_phaselock_all = []
    for i,b in batch.items():
        phases_modulations_all.append(b[0][1])
        metadata_phaselock_all.append(b[1])

    phases = b[0][0]
    phases_modulations_all = np.hstack(phases_modulations_all)
    metadata_phaselock_all = pd.concat(metadata_phaselock_all)

    return phases, phases_modulations_all,metadata_phaselock_all

In [32]:
# batch = bk.load.batch(main)
p,phase_mod,metadata = merge(batch)
np.sum(metadata.modulated[(metadata.Region == 'BLA') &  (metadata.Type == 'Pyr')])/np.sum((metadata.Region == 'BLA') & (metadata.Type == 'Pyr')) * 100

13.377609108159394

In [23]:
states = bk.load.states()
# lfp = bk.load.lfp_in_intervals(bk.load.random_channel('BLA'),states['Rem'])
lfp_filt = bk.signal.passband(lfp,6,10,1250,2)
power, phase = bk.signal.hilbert(lfp_filt)
amp = bk.signal.enveloppe(lfp_filt)

high_theta = treshold_to_intervals(tsd_zscore(amp),1,states['Rem'])

In [26]:
states = bk.load.states()
lfp_bla = bk.load.lfp_in_intervals(bk.load.random_channel('BLA'),states['Rem'])
lfp_filt_bla = bk.signal.passband(lfp_bla,6,10,1250,2)
power, phase = bk.signal.hilbert(lfp_filt_bla)
amp_bla = bk.signal.enveloppe(lfp_filt_bla)

high_theta_bla = treshold_to_intervals(tsd_zscore(amp_bla),1,states['Rem'])


In [24]:
%matplotlib qt

In [28]:
fig,ax = plt.subplots(2,1,sharex=True)
# ax[0].plot(power)
plt.sca(ax[0])
ax[0].plot(lfp.as_units('s'),'grey',alpha = 0.7)
ax[0].plot(lfp_filt.as_units('s'),'red',alpha = 0.4)
ax[0].plot(amp.as_units('s'),'orange',alpha = 0.5)
bk.plot.intervals(high_theta)
plt.ylim(-1000,1000)
plt.title('Hpc')



plt.sca(ax[1])
ax[1].plot(lfp_bla.as_units('s'),'grey',alpha = 0.7)
ax[1].plot(lfp_filt_bla.as_units('s'),'red',alpha = 0.4)
ax[1].plot(amp_bla.as_units('s'),'orange',alpha = 0.5)
bk.plot.intervals(high_theta_bla)
plt.ylim(-1000,1000)
plt.title('BLA')

Text(0.5, 1.0, 'BLA')

In [7]:
def treshold_to_intervals(signal,treshold,intersect = None):

    interval = np.int8(signal.values.T[0] > treshold)
    interval = np.append(interval,0)
    diff_interval = np.diff(interval)
    t_start = signal.index.values[diff_interval == 1]
    t_end = signal.index.values[diff_interval == -1]
    intervals = nts.IntervalSet(t_start,t_end)
    if intersect is not None:
        intervals = intervals.intersect(intersect)
    

    return intervals

In [9]:
def tsd_zscore(tsd,axis = 0):
    from scipy.stats import zscore
    t = tsd.index.values
    q = zscore(tsd.values,axis = axis)

    return nts.TsdFrame(t,q)

In [50]:
np.sum(states['Rem'].duration(time_units = 's'))*8 /2 *1250

11855000.0

In [117]:
lfp_hpc = bk.load.lfp(bk.load.ripple_channel(),0,6000)
lfp_hpc_filt = bk.signal.passband(lfp_hpc,5,10)
amp = bk.signal.enveloppe(lfp_hpc_filt)
plt.figure()
plt.hist(amp,5000)

(array([139., 320., 462., ...,   0.,   2.,   5.]),
 array([3.49266607e-02, 2.31647299e-01, 4.28367937e-01, ...,
        9.83244677e+02, 9.83441398e+02, 9.83638119e+02]),
 <BarContainer object of 5000 artists>)

In [64]:
plt.plot(lfp_hpc.as_units('s'),'grey',alpha = 0.8)
plt.plot(lfp_hpc_filt.as_units('s'),'red',alpha = 0.6)
plt.plot(amp.as_units('s'),'orange')
bk.plot.intervals(states['Rem'])

In [66]:
fig,ax = plt.subplots(2,1,sharex=True)


plt.sca(ax[1])
ax[1].plot(lfp_bla.as_units('s'),'grey',alpha = 0.7)
ax[1].plot(lfp_filt_bla.as_units('s'),'red',alpha = 0.4)
ax[1].plot(amp_bla.as_units('s'),'orange',alpha = 0.5)
bk.plot.intervals(high_theta_bla)
plt.ylim(-1000,1000)
plt.title('BLA')

Text(0.5, 1.0, 'BLA')

In [130]:
stru = ['Hpc','BLA']
channels = [bk.load.ripple_channel(),bk.load.random_channel('BLA')]

In [131]:
lfp = {}
ratio = {}
spectrogram = {}
for s,c in zip(stru,channels):
    lfp.update({s:bk.load.lfp(c,0,6000)})
    f,t,Sxx = scipy.signal.spectrogram(lfp[s].values,1250,nperseg = 1250)

    spectrogram.update({s:Sxx})
    filt_theta = (f>5) & (f<10)
    filt_delta = (f>0.5)& (f<4)

    theta = np.mean(Sxx[filt_theta],0)
    delta = np.mean(Sxx[filt_delta],0)

    ratio.update({s:theta/delta})